IMPORTED NECESSARY LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.layers import Conv1D, MaxPooling1D, SimpleRNN, LeakyReLU, Dense, Dropout, Flatten, LSTM, GRU
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error

READ THE DATA

In [ ]:
df = pd.read_csv('df-rajendranagar - 4year.csv')
#df = pd.read_csv('df-maruteru - 4year.csv')
#df = pd.read_csv('df-raipur - 3year.csv')
#df                                                                             #Uncomment to view the input dataset

CLIMATE FACTORS CALCULATED

In [ ]:
df['LTP'] = round(df['ssh'] * ((df['max_t'] + df['min_t']) / 2), 2)
df['TF'] = round(df['max_t'] - df['min_t'], 2)
df['PTR'] = round(df['rf'] / (df['max_t'] + df['min_t']), 2)
df['THC'] = round((df['rh1'] + df['rh2']) / (df['max_t'] + df['min_t']), 2)
#df                                                                             #Uncomment to view the input dataset

DIFFERENCED COLUMNS CREATED FOR THE RESIDUALS

In [ ]:
df_shifted = df.shift(1)
for col in df.columns:
    if col != 'date_time':  # Skip date_time column
        col_diff = col + '_diff'
        df[col_diff] = df[col] - df_shifted[col]
        df[col_diff].fillna(0, inplace=True)  # Replace NaN values with 0
#df                                                                             #Uncomment to view the input dataset

CREATE 1-LAG and 2-LAG TIME SHIFTED COLUMNS

In [ ]:
cols_to_shift = df.columns[1:]
for i in [1, 2]:
    shifted_df = df[cols_to_shift].shift(i)
    shifted_df.fillna(0, inplace=True)
    shifted_df.columns = [f"{col}+{i}" for col in shifted_df.columns]
    df = pd.concat([df, shifted_df], axis=1)
#df                                                                             #Uncomment to view the input dataset

In [ ]:
#df.info()                                                                      #Uncomment to view the details of each column types and values

CONVERT TO DATETIME DATATYPE

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'], infer_datetime_format=True)

CORRELATION MATRIX AND HEAT MAP FOR PEST WITH WEATHER

In [ ]:
# Calculate correlation coefficients between the columns of the dataframe df
corr_matrix = df.corr()

# Select the correlation values of the first column of the correlation matrix
corr_with_first_col = corr_matrix.iloc[:, 0]

# Set the figure size
plt.figure(figsize=(10,20))

# Generate a heatmap of the correlation values between the first column and all other columns
sns.heatmap(pd.DataFrame(corr_with_first_col), cmap='coolwarm', annot=True)
plt.title('Correlation with the First Column')
plt.show()

REMOVE UNRELATED COLUMNS - value 0.15 can be tuned to increase / decrease the number of columns being fed to the model for training

In [ ]:
df_input = df.iloc[:, 1:]     #df_input = df.iloc[:, 1:9]

#Only select columns with high correlation with the pest
high_corr_cols = corr_matrix.iloc[0][((corr_matrix.iloc[0] >= 0.15) | (corr_matrix.iloc[0] <= -0.15))].index.tolist()
df_input = df[high_corr_cols]

#df_input                                                                       #Uncomment to view the input dataset

In [ ]:
df_input.describe()

PEST DIFF and PEST LAG COLUMNS REMOVED, AS TARGET IS ONLY PEST AND INPUT IS ALL COLS EXCEPT PEST

In [ ]:
columns_to_drop = ['pest+1', 'pest+2', 'pest_diff', 'pest_diff+1', 'pest_diff+2']
columns_existing = [col for col in columns_to_drop if col in df_input.columns]

if columns_existing:
    df_input.drop(columns_existing, axis=1, inplace=True)

#df_input                                                                       #Uncomment to view the input dataset

PEST, PEST4W, PEST8W COLS CREATED AS TARGETS TO PREDICT 1, 4, 8 WEEKS INTO THE FUTURE

In [ ]:
df2 = pd.DataFrame()
df2['pest'] = df_input['pest'].copy()
df_input['pest'] = df_input['pest'].shift(-1)
df_input.insert(1, 'pest4w', df_input['pest'].shift(-3))
df_input.insert(2, 'pest8w', df_input['pest4w'].shift(-4))

df_input.loc[df_input.index[-1], 'pest'] = 0
df_input.loc[df_input.index[-4:], 'pest4w'] = 0
df_input.loc[df_input.index[-8:], 'pest8w'] = 0

#df_input                                                                       #Uncomment to view the input dataset

NORMALIZE THE DATA USING MIN MAX SCALER

In [ ]:
# Scaling and Normalizing
scaler = MinMaxScaler()  # Create a MinMaxScaler object

print(df_input.shape)  # Print the shape of the original DataFrame

# Compute the min and max values of each column of df_input and scale the values of each column to the range [0, 1]
# Assign the scaled data to the variable data_scaled
data_scaled = scaler.fit_transform(df_input)

# Print the resulting shape of the scaled data
print(data_scaled.shape)

#data_scaled                                                                    #Uncomment Print the scaled data

WEATHER PARAMETERS ADDED TO INPUT AND PEST TO TARGET

In [ ]:
#INPUT
features = data_scaled[:, 3:]               #This takes all the columns as input expect the pest column

#TARGET
target = data_scaled[:, 0:3]

print(features.shape)
print(target.shape)

DATASET SPLIT INTO TRAINING VALIDATION AND TESTING

In [ ]:
#Split the input data and target data into training, validation and testing sets.

test_size = 52 / len(features) # calculate the test size as a fraction of the total rows
x_train_val, x_test, y_train_val, y_test = train_test_split(features, target, test_size=test_size, shuffle=False)

train_size = 0.9 # set the size of the train set as a fraction of the remaining rows
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, train_size=train_size, shuffle=False)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

WINDOW LENGTH CAN BE MODIFIED BASED ON REQUIREMENT (WE TRIED WITH 4 and 2 Weeks as inputs)
KERNEL SIZE OF MAX POOL LAYER OF 1D CNN WILL DEPEND ON THE WIN_LENGTH

In [ ]:
win_length = 4                                                      #win_length = int(x_train.shape[0] / 32)        #win_length=8
batch_size = int(x_train.shape[0] / 4)                             #batch_size=4
num_features = x_train.shape[1]                                     #num_features=8 #This is to be selected based on columns of input
train_generator = TimeseriesGenerator(x_train, y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
val_generator = TimeseriesGenerator(x_val, y_val, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(x_test, y_test, length=win_length, sampling_rate=1, batch_size=batch_size)
print(win_length)
print(batch_size)
print(num_features)

FROM MODEL A/B, C/D, E/F - KEEP ONLY 1 CODE ACTIVE
>>> USE KERNEL SIZE = 3 and POOL SIZE = 2 FOR WINDOW LENGTH = 4
>>> USE KERNEL SIZE = 2 and POOL SIZE = 1 FOR WINDOW LENGHT = 2
>>>>> FOR OTHER WINDOW SIZE USE APPROPRIATE KERNEL AND POOL SIZE

ONLY MODEL A OR MODEL B TO BE SELECTED
1.   MODEL A: CNN-RNN (UNCOMMENT CNN LAYERS AND COMMENT THE FIRST RNN LAYER)
2.   MODEL B: RNN (COMMENT CNN LAYERS AND UNCOMMENT THE FIRST RNN LAYER)

In [ ]:
# Create the sequential model
model = Sequential()

#KEEP NEXT 3 LINES FOR THE MODEL A
# CNN Layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(win_length, num_features)))
model.add(MaxPooling1D(pool_size=2))

#KEEP NEXT 3 LINES FOR THE MODEL B
# # RNN Layers
# model.add(SimpleRNN(32, return_sequences=True, input_shape=(win_length, num_features)))
# model.add(LeakyReLU())

#NO CHANGE IN BELOW CODE
# RNN Layers
model.add(SimpleRNN(32, return_sequences=True))
model.add(LeakyReLU())

# Dense Layers
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='relu'))

# Print the model summary
model.summary()

ONLY MODEL C OR MODEL D TO BE SELECTED
1.   MODEL C: CNN-LSTM (UNCOMMENT CNN LAYERS AND COMMENT THE FIRST LSTM LAYER)
2.   MODEL D: LSTM (COMMENT CNN LAYERS AND UNCOMMENT THE FIRST LSTM LAYER)

In [ ]:
# # Create the sequential model
# model = Sequential()

# #KEEP NEXT 3 LINES FOR THE MODEL C
# #CNN Layers
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(win_length, num_features)))
# model.add(MaxPooling1D(pool_size=2))

# #KEEP NEXT 3 LINES FOR THE MODEL D
# # #LSTM Layers
# # model.add(LSTM(32, return_sequences=True, input_shape=(win_length, num_features)))
# # model.add(LeakyReLU())

# #LSTM Layers
# model.add(LSTM(32, return_sequences=True))
# model.add(LeakyReLU())

# #Dense Layers
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(3, activation='relu'))
# model.summary()

ONLY MODEL E OR MODEL F TO BE SELECTED
1.   MODEL E: CNN-GRU (UNCOMMENT CNN LAYERS AND COMMENT THE FIRST GRU LAYER)
2.   MODEL F: GRU (COMMENT CNN LAYERS AND UNCOMMENT THE FIRST GRU LAYER)

In [ ]:
# # Create the sequential model
# model = Sequential()

# #KEEP NEXT 3 LINES FOR THE MODEL E
# #CNN Layers
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(win_length, num_features)))
# model.add(MaxPooling1D(pool_size=2))

# #KEEP NEXT 3 LINES FOR THE MODEL F
# # #GRU Layers
# # model.add(GRU(32, return_sequences=True, input_shape=(win_length, num_features)))
# # model.add(LeakyReLU())

# #GRU Layers
# model.add(GRU(32, return_sequences=True))
# model.add(LeakyReLU())

# #Dense Layers
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(3, activation='relu'))
# model.summary()

MODEL FITTING

In [ ]:
checkpoint_filepath = 'model_best.h5'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,save_best_only=True,monitor='val_loss',mode='min',verbose=0)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
history = model.fit(train_generator, epochs=300, validation_data=val_generator,callbacks=[model_checkpoint_callback])

In [ ]:
#Lowest Validation Epoch is considered for the model training
best_epoch = np.argmin(history.history['val_loss'])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Training vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

# add a vertical line to mark the epoch with the lowest validation loss
plt.axvline(x=best_epoch, color='r', linestyle='--')
plt.show()

MODEL ARCHITECTURE

In [ ]:
plot_model(model, show_shapes=True, show_dtype=False, show_layer_names=True, rankdir="TB", expand_nested=True, dpi=96, layer_range=None, show_layer_activations=True)

PREDICT FROM THE TEST

In [ ]:
model.evaluate(test_generator, verbose=0)
predictions=model.predict(test_generator)
predictions.shape[0]

#print(predictions)                                                             #Uncomment to check for values
#print(predictions.shape)                                                       #Uncomment to check for values

POST PROCESSING TO BRING DATA BACK TO ACTUAL VALUES

In [ ]:
df_pred=pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[win_length:])],axis=1)
df_pred
print(df_pred.shape)

In [ ]:
rev_trans=scaler.inverse_transform(df_pred) #X = X_scaled * (X_max - X_min) + X_min

In [ ]:
df_final=df_input[predictions.shape[0]*-1:]
df_final.count()

In [ ]:
df_final['prediction']=rev_trans[:,0]
df_final['prediction-4w']=rev_trans[:,1]
df_final['prediction-8w']=rev_trans[:,2]

In [ ]:
df_final

df_final['prediction-4w'] = df_final['prediction-4w'].shift(4)
df_final['prediction-4w'].iloc[:4] = 0

df_final['prediction-8w'] = df_final['prediction-8w'].shift(8)
df_final['prediction-8w'].iloc[:8] = 0

In [ ]:
df_final['pest'] = df2['pest']
df_final_forresult = df_final
df_final_forresult

LONG-TERM (52 WEEK) PREDICTION PLOT

In [ ]:
df_final_forresult = df_final.copy()

fig, axs = plt.subplots(figsize=(10, 7))
plt.scatter(df_final_forresult.index, df_final_forresult['pest'], label='Actual', marker='o', s=50)
plt.scatter(df_final_forresult.index, df_final_forresult['prediction'], label='Prediction', marker='o', s=50)
axs.set_title('Pest vs Prediction')
axs.set_xlabel('Week (No.)')
axs.set_ylabel('Pest (No.s)')
plt.grid(True, linestyle='--', alpha=0.7, which='both')
legend = plt.legend(fontsize='medium')
for legend_handle in legend.legendHandles:
    legend_handle.set_sizes([50])  # Adjust marker size
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.margins(0.05)
plt.show()

mse = mean_squared_error(df_final_forresult['pest'], df_final_forresult['prediction'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(df_final_forresult['pest'], df_final_forresult['prediction'])
r2 = r2_score(df_final_forresult['pest'], df_final_forresult['prediction'])
mae = mean_absolute_error(df_final_forresult['pest'], df_final_forresult['prediction'])
print(f"RMSE: {round(rmse, 2)}; R2: {round(r2, 2)}; MAE: {round(mae, 2)}")

SHORT-TERM (4 WEEK) PREDICTION PLOT

In [ ]:
df_final_forresult = df_final.copy()
df_final_forresult.drop(df_final_forresult.index[4:], inplace=True)

fig, axs = plt.subplots(figsize=(10, 7))
plt.scatter(df_final_forresult.index, df_final_forresult['pest'], label='Actual', marker='o', s=50)
plt.scatter(df_final_forresult.index, df_final_forresult['prediction'], label='Prediction', marker='o', s=50)
axs.set_title('Pest vs Prediction')
axs.set_xlabel('Week (No.)')
axs.set_ylabel('Pest (No.s)')
plt.grid(True, linestyle='--', alpha=0.7, which='both')
legend = plt.legend(fontsize='medium')
for legend_handle in legend.legendHandles:
    legend_handle.set_sizes([50])  # Adjust marker size
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.margins(0.05)
plt.show()

mse = mean_squared_error(df_final_forresult['pest'], df_final_forresult['prediction'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(df_final_forresult['pest'], df_final_forresult['prediction'])
r2 = r2_score(df_final_forresult['pest'], df_final_forresult['prediction'])
mae = mean_absolute_error(df_final_forresult['pest'], df_final_forresult['prediction'])
print(f"RMSE: {round(rmse, 2)}; R2: {round(r2, 2)}; MAE: {round(mae, 2)}")

CONSOLIDATED PLOTS FROM ABOVE

In [ ]:
best_epoch = np.argmin(history.history['val_loss'])

plt.figure(figsize=(8, 12), dpi=150)  # Set figsize and DPI
plt.subplot(3, 1, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training vs Validation Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.axvline(x=best_epoch, color='r', linestyle='--')

df_final_forresult = df_final.copy()
plt.subplot(3, 1, 2)
plt.scatter(df_final_forresult.index, df_final_forresult['pest'], label='Actual', marker='o', s=50)
plt.scatter(df_final_forresult.index, df_final_forresult['prediction'], label='Prediction', marker='o', s=50)
plt.title('Pest vs Prediction')
plt.xlabel('Week (No.)')
plt.ylabel('Pest (No.s)')
plt.grid(True, linestyle='--', alpha=0.7, which='both')
legend = plt.legend(fontsize='medium')
for legend_handle in legend.legendHandles:
    legend_handle.set_sizes([50])  # Adjust marker size
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.margins(0.05)

mse_b = mean_squared_error(df_final_forresult['pest'], df_final_forresult['prediction'])
rmse_b = np.sqrt(mse_b)
mae_b = mean_absolute_error(df_final_forresult['pest'], df_final_forresult['prediction'])
r2_b = r2_score(df_final_forresult['pest'], df_final_forresult['prediction'])
metrics_b = f"RMSE: {round(rmse_b, 2)}; R2: {round(r2_b, 2)}; MAE: {round(mae_b, 2)}"
plt.text(0.5, -0.2, metrics_b, transform=plt.gca().transAxes, fontsize=10, ha='center')

df_final_forresult.drop(df_final_forresult.index[4:], inplace=True)
plt.subplot(3, 1, 3)
plt.scatter(df_final_forresult.index, df_final_forresult['pest'], label='Actual', marker='o', s=50)
plt.scatter(df_final_forresult.index, df_final_forresult['prediction'], label='Prediction', marker='o', s=50)
plt.title('Pest vs Prediction')
plt.xlabel('Week (No.)')
plt.ylabel('Pest (No.s)')
plt.grid(True, linestyle='--', alpha=0.7, which='both')
legend = plt.legend(fontsize='medium')
for legend_handle in legend.legendHandles:
    legend_handle.set_sizes([50])  # Adjust marker size
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.margins(0.05)

mse_c = mean_squared_error(df_final_forresult['pest'], df_final_forresult['prediction'])
rmse_c = np.sqrt(mse_c)
mae_c = mean_absolute_error(df_final_forresult['pest'], df_final_forresult['prediction'])
r2_c = r2_score(df_final_forresult['pest'], df_final_forresult['prediction'])
metrics_c = f"RMSE: {round(rmse_c, 2)}; R2: {round(r2_c, 2)}; MAE: {round(mae_c, 2)}"
plt.text(0.5, -0.2, metrics_c, transform=plt.gca().transAxes, fontsize=10, ha='center')

# Adjust subplot spacing
plt.tight_layout(pad=2)

# Save the plot to a file
plt.savefig('model_performance_plot.png', dpi=150, bbox_inches='tight')

plt.show()